In [57]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier
from sklearn.datasets import load_iris
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score

# Create models for DecisionTreeClassifier

When the max depth of the tree is restricted to 1, the accuracy decreases to around 66%. This is because at max_depth=1, the tree can only have 2 branches and therefore is only able to predict 2 out of the 3 classes in the dataset.

In [58]:
iris = load_iris()
X, y = shuffle(iris.data, iris.target)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
model = DecisionTreeClassifier().fit(X_train, y_train)
depth_model = DecisionTreeClassifier(max_depth=1).fit(X_train, y_train)

print(f"Standard: {accuracy_score(y_test, model.predict(X_test))}")
print(f"Limited depth: {accuracy_score(y_test, depth_model.predict(X_test))}")

Standard: 0.9555555555555556
Limited depth: 0.6666666666666666


# Create bagged and boosted models where max_depth=1, then cross-validate to measure performance
Although both models perform well when max_depth=1, the boosted method performs better than the bagged method. This is likely because since each tree can only seperate 2 classes, the boost classifer makes the other trees focus on the classes the previous tree could not detect, thus maintaining coverage of all 3 classes.

In comparison, the bagged model would have to depend on the random distribution of each subset to ensure that some trees cover classes not covered by others. Often, none of the the bagged trees will correctly identify the third class.

In [59]:
bag_model = BaggingClassifier(base_estimator=DecisionTreeClassifier(max_depth=1), n_estimators=5).fit(X, y)
boost_model = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1), n_estimators=5).fit(X, y)

print(f"Bagged: {cross_val_score(bag_model, X_train, y_train).mean()}")
print(f"Boosted: {cross_val_score(boost_model, X_train, y_train).mean()}")

Bagged: 0.7714285714285715
Boosted: 0.9333333333333333


# Compare all 4 models

I chose to use precision because given 3 classes, it is easy to see which classes perform poorly and how that poor performance affects the other classification scores due to how decision trees work. For example, the precision score for the max_depth=1 tree makes it obvious that 2 classes were being lumped into 1 group, and you can tell how they were lumped together because of how precision accounts for false positives.

Using a different metric, such as an accuracy score, would not be able to capture the nuances of 3 classes. Previously in this notebook, accuracy scores were used which did not make it clear how each class was being predicted.

In [60]:
print("Standard Decision Tree:")
print(confusion_matrix(y_test, model.predict(X_test)))
print(f"Precision: {precision_score(y_test, model.predict(X_test), average=None)}")

print("Small Decision Tree:")
print(confusion_matrix(y_test, depth_model.predict(X_test)))
print(f"Precision: {precision_score(y_test, depth_model.predict(X_test), average=None)}")

print("Bagged Decision Trees:")
print(confusion_matrix(y_test, bag_model.predict(X_test)))
print(f"Precision: {precision_score(y_test, bag_model.predict(X_test), average=None)}")

print("Boosted Decision Trees:")
print(confusion_matrix(y_test, boost_model.predict(X_test)))
print(f"Precision: {precision_score(y_test, boost_model.predict(X_test), average=None)}")

Standard Decision Tree:
[[16  0  0]
 [ 0 13  1]
 [ 0  1 14]]
Precision: [1.         0.92857143 0.93333333]
Small Decision Tree:
[[16  0  0]
 [ 0 14  0]
 [ 0 15  0]]
Precision: [1.         0.48275862 0.        ]
Bagged Decision Trees:
[[16  0  0]
 [ 0 14  0]
 [ 0  0 15]]
Precision: [1. 1. 1.]
Boosted Decision Trees:
[[16  0  0]
 [ 0 14  0]
 [ 0  0 15]]
Precision: [1. 1. 1.]


C:\Users\yinl3\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
